Nome: Gabriel Meirelles Carvalho Orlando
RA: 790728


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Data set utilizado: O data set deve ser transofrmado em csv

http://archive.ics.uci.edu/ml/datasets/Dry+Bean+Dataset#


O arquivo transformado em csv, está neste link

https://drive.google.com/file/d/15s0AD5RLoYTCcn5pH2a34mCdFACn9792/view?usp=sharing

In [10]:
# Vamos importar as bibliotecas necessárias
import pandas as pd
import graphviz
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split 
from sklearn import metrics 
from sklearn.tree import export_graphviz
from sklearn.model_selection import cross_val_score # Cross Validation Function.
from sklearn.model_selection import KFold # KFold Class.
from sklearn.model_selection import cross_validate

In [11]:
# Vamos criar uma função para visualizar nossa árvore
def plot_tree(tree, plot_title,feature_cols):

  # Dados do gráfico
  graph_data = export_graphviz(tree,
                               feature_names=feature_cols,
                               #Nomes das classes
                               class_names=['Seker', 'Barbunya', 'Bombay',' Cali', 'Dermosan', 'Horoz', 'Sira'],
                               filled=True,
                               rounded=True,
                               out_file=None)
  # Gerando o gráfico
  graph = graphviz.Source(graph_data)
  graph.render(plot_title)

  return graph

In [12]:
# Importação
pima = pd.read_csv("Dry_Bean_Dataset.csv", header=0)

In [13]:
#Dataframe
pima

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,28395,610.291,208.178117,173.888747,1.197191,0.549812,28715,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724,SEKER
1,28734,638.018,200.524796,182.734419,1.097356,0.411785,29172,191.272750,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430,SEKER
2,29380,624.110,212.826130,175.931143,1.209713,0.562727,29690,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066,SEKER
3,30008,645.884,210.557999,182.516516,1.153638,0.498616,30724,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199,SEKER
4,30140,620.134,201.847882,190.279279,1.060798,0.333680,30417,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166,SEKER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13606,42097,759.696,288.721612,185.944705,1.552728,0.765002,42508,231.515799,0.714574,0.990331,0.916603,0.801865,0.006858,0.001749,0.642988,0.998385,DERMASON
13607,42101,757.499,281.576392,190.713136,1.476439,0.735702,42494,231.526798,0.799943,0.990752,0.922015,0.822252,0.006688,0.001886,0.676099,0.998219,DERMASON
13608,42139,759.321,281.539928,191.187979,1.472582,0.734065,42569,231.631261,0.729932,0.989899,0.918424,0.822730,0.006681,0.001888,0.676884,0.996767,DERMASON
13609,42147,763.779,283.382636,190.275731,1.489326,0.741055,42667,231.653248,0.705389,0.987813,0.907906,0.817457,0.006724,0.001852,0.668237,0.995222,DERMASON


Dataframe tem 13611 linhas, logo se tirar uma, a divisão por 10 fica exata, retira a útlima linha do dataframe

In [14]:
pima.drop(pima.index[13610],inplace=True)

Embaralha as linhas do dataframe, visto que classes do mesmo valor estavam ficando juntas, gerando erro no fold

In [15]:
pima = pima.sample(frac = 1)

Indexar corretamente o dataframe

In [16]:
pima = pima.set_index(pd.Index(range(0,13610)))


In [17]:
pima

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,34432,676.666,241.828841,181.660880,1.331210,0.660079,34759,209.380477,0.803735,0.990592,0.944982,0.865821,0.007023,0.002435,0.749646,0.997936,DERMASON
1,167563,1560.256,576.863825,372.761506,1.547541,0.763180,169787,461.895917,0.819704,0.986901,0.864960,0.800702,0.003443,0.000873,0.641123,0.992165,BOMBAY
2,169128,1572.037,602.537795,360.907045,1.669510,0.800765,171304,464.047905,0.770669,0.987297,0.860003,0.770156,0.003563,0.000773,0.593140,0.990253,BOMBAY
3,41287,761.858,278.168553,189.952797,1.464409,0.730540,41809,229.277651,0.721927,0.987515,0.893872,0.824240,0.006737,0.001918,0.679372,0.994878,SIRA
4,63234,1025.276,416.665186,194.557390,2.141606,0.884289,64172,283.746417,0.675187,0.985383,0.755925,0.680994,0.006589,0.000874,0.463753,0.993175,HOROZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13605,36998,714.759,262.337082,179.855847,1.458596,0.727987,37380,217.042200,0.745026,0.989781,0.910058,0.827341,0.007091,0.002049,0.684493,0.998399,DERMASON
13606,48889,826.058,316.859913,197.607830,1.603479,0.781708,49359,249.494305,0.756772,0.990478,0.900326,0.787396,0.006481,0.001537,0.619993,0.994145,SIRA
13607,45478,838.042,331.843270,176.276767,1.882513,0.847243,46206,240.633306,0.608011,0.984244,0.813729,0.725141,0.007297,0.001245,0.525830,0.989882,HOROZ
13608,81592,1092.977,432.959510,242.334919,1.786616,0.828684,82729,322.313762,0.808546,0.986256,0.858293,0.744443,0.005306,0.001005,0.554196,0.990135,CALI


Separar em matriz de atributos e vetor de classes


In [18]:
# Vamos separar as variáveis preditoras da variável label
final = pima.columns.shape[0]-1
feature_cols = pima.columns[0:final]
X = pima[feature_cols]
y = pima.Class 

Método que calcula os indices do kfold

In [19]:
import random
def kfoldcv(indices, k = 10):
    
    size = len(indices)
    subset_size = size // k
    subsets = [indices[x:x+subset_size] for x in range(0, len(indices), subset_size)]
    kfolds = []
    for i in range(k):
        test = subsets[i]
        train = []
        for subset in subsets:
            if subset != test:
                train.append(subset)
        kfolds.append((train,test))
        
    return kfolds

Faz o fold dos indices

In [20]:
indices = list(range(X.shape[0]))
k = 10
#indices = list(range(10))
kf = kfoldcv(indices,k)

In [ ]:
len(indices)

13610

União dos folds, gerando os conjuntos de treino

In [21]:
def Union_train(X,y,indexes,k,j):
  #X_train
  df = list(range(k-1))
  for p in range(0, k-1):
    df[p] = X[kf[j][0][p][0]:kf[j][0][p][-1]] 
  x_train = pd.concat(df)

  df2 = list(range(k-1))
  for p in range(0,k-1):
    df2[p] = y[kf[j][0][p][0]:kf[j][0][p][-1]]
  y_train = pd.concat(df2)


  return x_train,y_train

Gera 10 objetos da classe DecisionTreeClassifier

In [22]:
clf = list(range(k))
for i in range(0,k):
  clf[i] = DecisionTreeClassifier(criterion='entropy')

Cria listas, para guardar os valores

In [23]:
model = list(range(k))
y_pred = list(range(k))
y_test = list(range(k))

Executa a construção da arvore de decisão, e também ja calcula o y_pred baseado na arvore construida previamente

In [ ]:
for j in range(0,k):

  #Conjuntos de testes e de treinamentos
  X_train, y_train = Union_train(X,y,kf,k,j)
 # display(X_train)

  X_test = X[kf[j][1][0]:kf[j][1][-1]] 

  y_test[j] = y[kf[j][1][0]:kf[j][1][-1]] 

            

  model[j] = clf[j].fit(X_train,y_train)

  y_pred[j] = model[j].predict(X_test)

  


In [25]:
#Importa biblioteca numpy
import numpy as np

Verifica se cada y_pred contém dados de todas as classes

In [26]:
for u in range(0,k):
  print(np.unique(y_pred[u]))

['BARBUNYA' 'BOMBAY' 'CALI' 'DERMASON' 'HOROZ' 'SEKER' 'SIRA']
['BARBUNYA' 'BOMBAY' 'CALI' 'DERMASON' 'HOROZ' 'SEKER' 'SIRA']
['BARBUNYA' 'BOMBAY' 'CALI' 'DERMASON' 'HOROZ' 'SEKER' 'SIRA']
['BARBUNYA' 'BOMBAY' 'CALI' 'DERMASON' 'HOROZ' 'SEKER' 'SIRA']
['BARBUNYA' 'BOMBAY' 'CALI' 'DERMASON' 'HOROZ' 'SEKER' 'SIRA']
['BARBUNYA' 'BOMBAY' 'CALI' 'DERMASON' 'HOROZ' 'SEKER' 'SIRA']
['BARBUNYA' 'BOMBAY' 'CALI' 'DERMASON' 'HOROZ' 'SEKER' 'SIRA']
['BARBUNYA' 'BOMBAY' 'CALI' 'DERMASON' 'HOROZ' 'SEKER' 'SIRA']
['BARBUNYA' 'BOMBAY' 'CALI' 'DERMASON' 'HOROZ' 'SEKER' 'SIRA']
['BARBUNYA' 'BOMBAY' 'CALI' 'DERMASON' 'HOROZ' 'SEKER' 'SIRA']


Verifica se cada y_test contém dados de todas as classses

In [27]:
for u in range(0,k):
  print(np.unique(y_test[u]))

['BARBUNYA' 'BOMBAY' 'CALI' 'DERMASON' 'HOROZ' 'SEKER' 'SIRA']
['BARBUNYA' 'BOMBAY' 'CALI' 'DERMASON' 'HOROZ' 'SEKER' 'SIRA']
['BARBUNYA' 'BOMBAY' 'CALI' 'DERMASON' 'HOROZ' 'SEKER' 'SIRA']
['BARBUNYA' 'BOMBAY' 'CALI' 'DERMASON' 'HOROZ' 'SEKER' 'SIRA']
['BARBUNYA' 'BOMBAY' 'CALI' 'DERMASON' 'HOROZ' 'SEKER' 'SIRA']
['BARBUNYA' 'BOMBAY' 'CALI' 'DERMASON' 'HOROZ' 'SEKER' 'SIRA']
['BARBUNYA' 'BOMBAY' 'CALI' 'DERMASON' 'HOROZ' 'SEKER' 'SIRA']
['BARBUNYA' 'BOMBAY' 'CALI' 'DERMASON' 'HOROZ' 'SEKER' 'SIRA']
['BARBUNYA' 'BOMBAY' 'CALI' 'DERMASON' 'HOROZ' 'SEKER' 'SIRA']
['BARBUNYA' 'BOMBAY' 'CALI' 'DERMASON' 'HOROZ' 'SEKER' 'SIRA']


Importa biblioteca para sortear um numero aleatorio

In [28]:
from random import randint

Numero aleatorio, para exibição de um arvore de decisão

In [40]:
cod_arvore = randint(0,9)
cod_arvore

9

Plot da árvore de decisão

In [ ]:
plot_tree(model[cod_arvore],'Wine',feature_cols=feature_cols)

In [ ]:
#Importanto novas bibliotecas
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

Confusion matrix whose i-th row and j-th column entry indicates the number of samples with true label being i-th class and predicted label being j-th class.

[i][j] representa o numero de amostras com a classificação da classe i mas foi predicted da classe j

Função que avalia a matriz de confusão

In [ ]:
def evaluation(vector,classe):
  classes = ['Seker', 'Barbunya', 'Bombay',' Cali', 'Dermosan', 'Horoz', 'Sira']
  print(f"Classe {classes[classe]}:\n")
  print(f"{vector[classe]} amostras foram previstos certos.")
  soma = vector.sum()
  errados = abs(vector[classe] - soma)
  if(errados == 0):
    print("Todas as amostrar foram previstas corretamente\n")
  else:
    print(f"{errados} amostras foram previstas incorretamente\n")
  


Imprime o dicionário criado pela função classification report

In [ ]:
def print_dicionario(dicionario):

  nomes = []
  for k in dicionario.keys():
    nomes.append(k)
  
  print("           precision   recall    f1-score    support\n")
  for j in nomes:
    if(j != 'accuracy'):
      print(f"{j:>11}      {dicionario[j]['precision']:.2f}       {dicionario[j]['recall']:.2f}       {dicionario[j]['f1-score']:.2f}    {dicionario[j]['support']:.0f}")
    else:
      space = ' '
      print(f"{j}{31*space}{dicionario[j]:.2f}    {dicionario['macro avg']['support']:.0f}")

  print("\n\n")
  


Cria as listas que serão feitas as análises

In [ ]:
target_names = ['Seker', 'Barbunya', 'Bombay',' Cali', 'Dermosan', 'Horoz', 'Sira']
#Vetores para a analise
precisao = [[],[],[],[],[],[],[],[],[]]
recall = [[],[],[],[],[],[],[],[],[]]
f1_score = [[],[],[],[],[],[],[],[],[],[]]

Analise desejada, com a matriz de confusão e o classification report, junto a uma breve analise de cada método.

In [ ]:
for i in range(0,10):
  print(f"Para o Fold {i}:")

  c = confusion_matrix(y_test[i], y_pred[i])
  classification = classification_report(y_test[i], y_pred[i], target_names=target_names,output_dict=True)

 
  print_dicionario(classification)

  for l in range(0,7):
    evaluation(c[l],l)

  
 


  #Analise

  for p in range(0,7):
    precisao[p].append(classification[target_names[p]]['precision'])
    recall[p].append(classification[target_names[p]]['recall'])
    f1_score[p].append(classification[target_names[p]]['f1-score'])

  f1_score[7].append(classification['accuracy'])

  resto = ['macro avg','weighted avg']
  for p in range(8,10):
    precisao[p-1].append(classification[resto[p-8]]['precision'])
    recall[p-1].append(classification[resto[p-8]]['recall'])
    f1_score[p].append(classification[resto[p-8]]['f1-score'])
  

Transforma as arrays do python em arrays do numpy para facilitar o calculo de média e do std

In [ ]:
arr = np.array(precisao)
arr2 = np.array(recall)
arr3 = np.array(f1_score)

In [ ]:
rotulos = ['Seker', 'Barbunya', 'Bombay',' Cali', 'Dermosan', 'Horoz', 'Sira','accuracy','macro avg','weighted avg']

Imprime as medias, desvios padroões dos valores recebidos do método classification_report

In [ ]:
arrange = 0
for x in range(0,10):
  if(rotulos[x] != 'accuracy'):
    print(f"Media da precisão da {rotulos[x]}: {arr[x+arrange].mean()}")
    print(f"Desvio padrão da precisão da {rotulos[x]}: {arr[x+arrange].std()}\n")

    print(f"Media do recall da {rotulos[x]}: {arr2[x+arrange].mean()}")
    print(f"Desvio padrão do recall da {rotulos[x]}: {arr2[x+arrange].std()}\n")
  else:
    arrange = -1

  print(f"Media do f1-score da {rotulos[x]}: {arr3[x].mean()}")
  print(f"Desvio padrão do f1-score da {rotulos[x]}: {arr3[x].std()}\n")

  print("--------------------------------------------------------------------------")

Media da precisão da Seker: 0.8911128920209513
Desvio padrão da precisão da Seker: 0.018704966611244777

Media do recall da Seker: 0.8956840070288246
Desvio padrão do recall da Seker: 0.029682183537252105

Media do f1-score da Seker: 0.8929055749430779
Desvio padrão do f1-score da Seker: 0.013502381549313736

--------------------------------------------------------------------------
Media da precisão da Barbunya: 0.9981818181818183
Desvio padrão da precisão da Barbunya: 0.005454545454545457

Media do recall da Barbunya: 0.9983606557377049
Desvio padrão do recall da Barbunya: 0.0049180327868852515

Media do f1-score da Barbunya: 0.9982561225263478
Desvio padrão do f1-score da Barbunya: 0.0034936837068063956

--------------------------------------------------------------------------
Media da precisão da Bombay: 0.9215169614653294
Desvio padrão da precisão da Bombay: 0.02029848508817588

Media do recall da Bombay: 0.9107301006881491
Desvio padrão do recall da Bombay: 0.017730006864717516
